# Escolha do Algoritmo de Validação Cruzada

Neste notebook, selecionaremos o melhor método de validação cruzada para séries temporais, utilizando uma amostra do dataset do nosso projeto. Embora existam diversas estratégias de validação cruzada, neste estudo nos concentraremos em abordagens específicas para séries temporais, que respeitam a dependência temporal dos dados. Esse critério é essencial para dados de varejo, onde padrões sazonais e tendências ao longo do tempo influenciam significativamente as previsões.

In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

df_sales = pd.read_parquet("../data/processed/sales_cleaning_processed.parquet")

df_sales_filtrado = df_sales[(df_sales["store_id"] == "TX_1") & (df_sales["cat_id"] == "FOODS")]
df_sales_filtrado

,store_id,cat_id,date,sales,in_training
21708,TX_1,FOODS,2011-01-29,3950.35,True
21709,TX_1,FOODS,2011-01-30,3844.97,True
21710,TX_1,FOODS,2011-01-31,2888.03,True
21711,TX_1,FOODS,2011-02-01,3631.28,True
21712,TX_1,FOODS,2011-02-02,3072.18,True
...,...,...,...,...,...
23644,TX_1,FOODS,2016-05-18,5146.18,True
23645,TX_1,FOODS,2016-05-19,4647.91,True
23646,TX_1,FOODS,2016-05-20,5977.50,True
23647,TX_1,FOODS,2016-05-21,6776.10,True


# Prequential Expanding Split

A estratégia de validação **Prequential Expanding Split** é uma abordagem baseada na divisão progressiva dos dados em **treino, validação e teste**, garantindo que o modelo seja avaliado de forma realista ao longo do tempo. Essa técnica é útil para cenários onde os dados evoluem continuamente, permitindo que o modelo se ajuste conforme novas informações se tornam disponíveis.

## Como funciona a Validação Prequential Expanding?

1. **Divisão dos Dados em Blocos**  
   - O conjunto de dados é dividido em `n_block` partes sequenciais.  
   - Cada bloco representa um período ou um segmento contínuo dos dados.

2. **Definição dos Conjuntos**  
   - Os primeiros blocos são usados para o treinamento do modelo.  
   - O penúltimo bloco (`n_block - 2`) é reservado para a validação.  
   - O último bloco (`n_block - 1`) é usado como teste.

3. **Opção de Gap**  
   - Se `gap=True`, há um espaço entre os conjuntos de treino e validação, garantindo que o modelo não treine diretamente nos dados mais próximos da validação e do teste.

4. **Expansão Gradual**  
   - Conforme mais dados se tornam disponíveis, o conjunto de treinamento cresce progressivamente.

---

## Vantagens da Abordagem Prequential Expanding

- **Adaptação contínua**: permite que o modelo capture padrões que mudam ao longo do tempo.  
- **Evita vazamento de dados**: a separação correta dos conjuntos previne a contaminação por informações futuras.  
- **Reflete cenários reais de produção**: simula o fluxo contínuo de dados em sistemas dinâmicos.  


In [2]:
def prequential_expanding_split(df, n_block, validation=True, gap=False):
    """
    Implementa a divisão dos dados em treino, validação e teste seguindo o esquema prequential expanding window.

    Parameters:
    - df: DataFrame contendo os dados, incluindo as features e a variável target.
    - n_block: Número de blocos para dividir os dados.
    - gap: Se True, adiciona um espaço entre treino e validação para evitar data leakage.

    Returns:
    - training_df: Dados de treino (janela crescente).
    - validation_df: Dados de validação.
    - test_df: Dados de teste.
    """
    df = df.copy()

    df = df.reset_index(drop=False)

    # Criar 'series_id' se necessário
    if "series_id" not in df.columns:
        df = df.assign(series_id=df["store_id"] + "_" + df["cat_id"])

    test_list, validation_list, training_list = [], [], []

    # Iterar sobre cada série única
    for series in df["series_id"].unique():
        df_series = df.loc[df["series_id"] == series]
        blocos = np.array_split(df_series.reset_index(), n_block)

        for i, bloco in enumerate(blocos):
            bloco["block"] = i

        novo_df = pd.concat(blocos, ignore_index=True).set_index("index")

        # Definição dos blocos de treino, validação e teste
        test_block = n_block - 1 
        val_block = n_block - 2
        train_blocks = list(range(val_block - (1 if gap else 0)))

        df_series_test = novo_df[novo_df["block"] == test_block].drop(columns="block")

        if validation:
            df_series_val = novo_df[novo_df["block"] == val_block].drop(columns="block")
            df_series_train = novo_df[novo_df["block"].isin(train_blocks)].drop(columns="block")
        else:
            df_series_val = pd.DataFrame()
            df_series_train = novo_df[novo_df["block"].isin(train_blocks)].drop(columns="block")

        # Adicionar aos conjuntos
        test_list.append(df_series_test)
        validation_list.append(df_series_val)
        training_list.append(df_series_train)

    # Concatenar os DataFrames resultantes
    return (
        pd.concat(training_list, ignore_index=True),
        pd.concat(validation_list, ignore_index=True) if validation else pd.DataFrame(),
        pd.concat(test_list, ignore_index=True),
    )


# Prequential Sliding Split

O método de validação **Prequential Sliding Split** é uma técnica de **validação cruzada para fluxos de dados contínuos**, projetada para avaliar a performance de modelos que precisam se adaptar constantemente. Ao contrário da validação tradicional, onde o conjunto de treinamento cresce indefinidamente, nesta abordagem a janela de treino desliza ao longo do tempo, garantindo que o modelo esteja sempre treinando nos dados mais recentes.

## Como funciona a Validação Prequential Sliding?

1. **Divisão dos Dados em Blocos**  
   - O conjunto de dados é segmentado em `n_block` partes sequenciais.  
   - Cada bloco representa um período ou um conjunto contínuo de observações.

2. **Janela Deslizante (Sliding Window)**  
   - O modelo é treinado apenas com o bloco mais recente antes da validação.  
   - Diferente da abordagem **Expanding**, onde o conjunto de treino cresce, aqui ele se move para frente, mantendo um tamanho fixo.

3. **Avaliação Dinâmica**  
   - A cada novo bloco, o modelo é treinado em um bloco e avaliado no próximo.  
   - O penúltimo bloco (`n_block - 2`) é reservado para validação.  
   - O último bloco (`n_block - 1`) é usado como teste final.

4. **Adaptação a Mudanças nos Dados**  
   - Como o conjunto de treinamento é sempre atualizado, o modelo se ajusta rapidamente a mudanças na distribuição dos dados, como variações sazonais e mudanças de regime.

---

## Vantagens da Validação Prequential Sliding

- **Melhor para dados não estacionários**: útil para aplicações onde os padrões mudam rapidamente, como mercado financeiro e detecção de fraudes.  
- **Evita o aprendizado de padrões obsoletos**: apenas os dados mais recentes são utilizados para treino.  
- **Aplicável a cenários de streaming de dados**: ideal para aprendizado online e adaptação contínua.  


In [3]:
import numpy as np
import pandas as pd

df_sales = pd.read_parquet("../data/processed/sales_cleaning_processed.parquet")

def prequential_sliding_split(df, n_block, validation=True, gap=False):
    """
    Implementa a divisão dos dados em treino, validação e teste seguindo o esquema prequential sliding window.

    Parameters:
    - df: DataFrame contendo os dados, incluindo as features e a variável target.
    - n_block: Número de blocos para dividir os dados.
    - gap: Se True, adiciona um espaço entre treino e validação para evitar data leakage.

    Returns:
    - training_df: Dados de treino (apenas um bloco).
    - validation_df: Dados de validação (próximo bloco após treino).
    - test_df: Dados de teste (último bloco).
    """
    # Criar uma cópia para evitar modificar o DataFrame original
    df = df.copy()

    df = df.reset_index(drop=False)

    # Criar 'series_id' se necessário
    if "series_id" not in df.columns:
        df = df.assign(series_id=df["store_id"] + "_" + df["cat_id"])

    test_list, validation_list, training_list = [], [], []

    # Iterar sobre cada série única
    for series in df["series_id"].unique():
        df_series = df.loc[df["series_id"] == series]
        blocos = np.array_split(df_series.reset_index(), n_block)

        for i, bloco in enumerate(blocos):
            bloco["block"] = i

        novo_df = pd.concat(blocos, ignore_index=True).set_index("index")

        # Definição dos blocos de treino, validação e teste
        test_block = n_block - 1
        val_block = n_block - 2
        train_block = val_block - 1 if not gap else val_block - 2

        df_series_test = novo_df[novo_df["block"] == test_block].drop(columns="block")

        if validation:
            df_series_val = novo_df[novo_df["block"] == val_block].drop(columns="block")
            df_series_train = novo_df[novo_df["block"] == train_block].drop(columns="block")
        else:
            df_series_val = pd.DataFrame()
            df_series_train = novo_df[novo_df["block"] == train_block].drop(columns="block")

        # Adicionar aos conjuntos
        test_list.append(df_series_test)
        validation_list.append(df_series_val)
        training_list.append(df_series_train)

    # Concatenar os DataFrames resultantes
    return (
        pd.concat(training_list, ignore_index=True),
        pd.concat(validation_list, ignore_index=True) if validation else pd.DataFrame(),
        pd.concat(test_list, ignore_index=True),
    )


# Expanding Window Split

O método de validação **Expanding Window Split** é uma técnica de **validação cruzada para séries temporais**, onde o tamanho do conjunto de treinamento cresce progressivamente a cada iteração. Essa abordagem é útil para modelagem preditiva em cenários onde novos dados se tornam disponíveis continuamente, permitindo que o modelo aprenda com um histórico cada vez maior.

## Como funciona a Validação Expanding Window?

1. **Divisão dos Dados em Blocos**  
   - O conjunto de dados é segmentado em `n_block` partes sequenciais.  
   - Cada bloco representa um período contínuo de observações.

2. **Janela Expansiva (Expanding Window)**  
   - O modelo é treinado em todos os blocos anteriores à validação.  
   - A cada nova iteração, o conjunto de treinamento cresce, incorporando mais dados históricos.

3. **Avaliação Dinâmica**  
   - O penúltimo bloco (`n_block - 2`) é reservado para validação.  
   - O último bloco (`n_block - 1`) é usado como teste final.

4. **Adaptação a Mudanças nos Dados**  
   - Como o conjunto de treinamento aumenta progressivamente, o modelo se ajusta melhor a tendências de longo prazo, capturando padrões históricos de forma mais robusta.

---

## Vantagens da Validação Expanding Window

- **Aprendizado progressivo**: permite que o modelo utilize um histórico crescente de dados, melhorando a estabilidade das previsões.  
- **Ideal para séries temporais**: mantém a ordem cronológica dos dados, essencial para previsões realistas.  
- **Reflete aplicações do mundo real**: útil para modelos que devem considerar tendências de longo prazo.  


In [4]:
def expanding_window_split(df, split_num, prediction_length=28, validation=True):
    """
    Implementa a divisão dos dados em treino, validação e teste seguindo uma estratégia de cross-validation.

    Parameters:
    - df: DataFrame contendo as séries temporais, incluindo as colunas 'store_id', 'cat_id' e 'date'.
    - split_num: Número do split para controle do deslocamento das janelas de treino e teste.
    - prediction_length: Número de dias para cada janela de previsão.
    - validation: Booleano indicando se deve incluir um conjunto de validação.

    Returns:
    - training_df: DataFrame com os dados de treino.
    - validation_df: DataFrame com os dados de validação (se `validation=True`).
    - test_df: DataFrame com os dados de teste.
    """

    # Criar uma cópia para evitar modificar o DataFrame original
    df = df.copy()

    df = df.reset_index(drop=False)

    # Criar 'series_id' se necessário
    if "series_id" not in df.columns:
        df = df.assign(series_id=df["store_id"] + "_" + df["cat_id"])

    test_list, validation_list, training_list = [], [], []

    # Calcular deslocamentos de datas baseados em `split_num`
    test_offset = prediction_length * ((split_num + 1) * 2 - 1)
    val_offset = prediction_length * (split_num + 1) * 2
    test_upper_offset = prediction_length * (split_num * 2)

    # Iterar sobre cada série única
    for series in df["series_id"].unique():
        df_series = df.loc[df["series_id"] == series]
        max_date = df_series["date"].max()
        min_date = df_series["date"].min()

        # Calcular limites de datas para cada conjunto
        test_lower_date = max_date - pd.Timedelta(f"{test_offset} days")
        test_upper_date = max_date - pd.Timedelta(f"{test_upper_offset} days")
        val_lower_date = max_date - pd.Timedelta(f"{val_offset} days")

        # Verificar se há dados suficientes para a divisão
        if min(test_lower_date, test_upper_date) < min_date:
            raise ValueError(
                f"A série '{series}' não possui dados suficientes para o split {split_num}. "
                f"Data mínima: {min_date}, necessário: {test_lower_date}."
            )

        # Aplicar filtros para separar os conjuntos de dados
        df_series_test = df_series.query("date > @test_lower_date and date <= @test_upper_date")
        df_series_val = (
            df_series.query("date > @val_lower_date and date <= @test_lower_date")
            if validation
            else pd.DataFrame()
        )
        df_series_train = (
            df_series.query("date <= @val_lower_date")
            if validation
            else df_series.query("date <= @test_lower_date")
        )

        # Adicionar aos conjuntos
        test_list.append(df_series_test)
        validation_list.append(df_series_val)
        training_list.append(df_series_train)

    # Concatenar os DataFrames resultantes
    return (
        pd.concat(training_list, ignore_index=True),
        pd.concat(validation_list, ignore_index=True) if validation else pd.DataFrame(),
        pd.concat(test_list, ignore_index=True),
    )


Para nosso projeto, realizaremos um teste para garantir que a escolha da estratégia de cross-validation seja baseada no melhor resultado e em sua aplicabilidade prática. Para isso, criaremos um modelo simples de previsão de vendas e avaliaremos o desempenho de cada uma das abordagens de validação cruzada.

In [7]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error


# Função para calcular as métricas de avaliação
def avaliar_metricas(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    # Evita divisão por zero ao calcular o MAPE
    y_true_nonzero = y_true[y_true != 0]
    y_pred_nonzero = y_pred[y_true != 0]
    if len(y_true_nonzero) > 0:
        mape = np.mean(np.abs((y_true_nonzero - y_pred_nonzero) / y_true_nonzero)) * 100
    else:
        mape = np.nan
    return {"MAE": mae, "MSE": mse, "RMSE": rmse, "MAPE": mape}


# Função para previsão com ARIMA
def arima_forecast(train, steps, order=(1, 1, 1)):
    """
    Ajusta um modelo ARIMA na série de treino e prevê os próximos 'steps' períodos.
    """
    # Ordena os dados pela coluna 'date'
    train = train.sort_values("date")
    modelo = ARIMA(train["sales"], order=order)
    modelo_fit = modelo.fit()
    previsao = modelo_fit.forecast(steps=steps)
    return previsao.values


# Função auxiliar para aplicar o split, ajustar ARIMA e avaliar as métricas
def avaliar_split_arima(split_func, df, forecast_order=(1, 1, 1), **kwargs):
    # Realiza a divisão dos dados
    train_df, val_df, test_df = split_func(df, **kwargs)

    if len(train_df) == 0:
        raise ValueError("Não há dados suficientes no conjunto de treino.")

    # Previsão e avaliação para o conjunto de validação (se existir)
    if not val_df.empty:
        y_val_true = val_df["sales"].values
        try:
            y_val_pred = arima_forecast(train_df, len(val_df), order=forecast_order)
        except Exception as e:
            print("Erro ao ajustar ARIMA na validação:", e)
            y_val_pred = np.full(len(val_df), np.nan)
        val_metrics = avaliar_metricas(y_val_true, y_val_pred)
    else:
        val_metrics = {"MAE": np.nan, "MSE": np.nan, "RMSE": np.nan, "MAPE": np.nan}

    # Re-treina o modelo usando treino + validação para previsão no teste
    full_train_val = pd.concat([train_df, val_df], ignore_index=True).sort_values("date")
    if len(full_train_val) == 0:
        raise ValueError("Não há dados suficientes para re-treino.")
    y_test_true = test_df["sales"].values
    try:
        y_test_pred = arima_forecast(full_train_val, len(test_df), order=forecast_order)
    except Exception as e:
        print("Erro ao ajustar ARIMA no teste:", e)
        y_test_pred = np.full(len(test_df), np.nan)
    test_metrics = avaliar_metricas(y_test_true, y_test_pred)

    return {"val_metrics": val_metrics, "test_metrics": test_metrics}


# Supondo que o DataFrame 'df_sales_filtrado' já esteja carregado e filtrado:
df = df_sales_filtrado.copy()

resultados_arima = {}

# 1) Prequential Expanding Split com ARIMA
resultados_arima["prequential_expanding"] = avaliar_split_arima(
    split_func=prequential_expanding_split,
    df=df,
    n_block=3,
    validation=True,
    gap=False,
    forecast_order=(1, 1, 1),
)

# 2) Prequential Sliding Split com ARIMA
resultados_arima["prequential_sliding"] = avaliar_split_arima(
    split_func=prequential_sliding_split,
    df=df,
    n_block=3,
    validation=True,
    gap=False,
    forecast_order=(1, 1, 1),
)

# 3) Expanding Window Split com ARIMA
resultados_arima["expanding_window"] = avaliar_split_arima(
    split_func=expanding_window_split,
    df=df,
    split_num=3,
    prediction_length=28,
    validation=True,
    forecast_order=(1, 1, 1),
)

# Exibindo os resultados de forma organizada
for metodo, valores in resultados_arima.items():
    print(f"\nMétodo de Split: {metodo}")
    print(" - Métricas Validação:", valores["val_metrics"])
    print(" - Métricas Teste:    ", valores["test_metrics"])


Método de Split: prequential_expanding
 - Métricas Validação: {'MAE': 748.7203346256707, 'MSE': 931445.955538253, 'RMSE': np.float64(965.1144779445872), 'MAPE': np.float64(15.304957661140847)}
 - Métricas Teste:     {'MAE': 750.6894226595261, 'MSE': 880171.306406956, 'RMSE': np.float64(938.1744541432345), 'MAPE': np.float64(14.494219661012586)}

Método de Split: prequential_sliding
 - Métricas Validação: {'MAE': 748.7203346256707, 'MSE': 931445.955538253, 'RMSE': np.float64(965.1144779445872), 'MAPE': np.float64(15.304957661140847)}
 - Métricas Teste:     {'MAE': 750.6894226595261, 'MSE': 880171.306406956, 'RMSE': np.float64(938.1744541432345), 'MAPE': np.float64(14.494219661012586)}

Método de Split: expanding_window
 - Métricas Validação: {'MAE': 676.6035249467376, 'MSE': 611017.3399737701, 'RMSE': np.float64(781.6759814486883), 'MAPE': np.float64(15.077434220983314)}
 - Métricas Teste:     {'MAE': 798.6517968854467, 'MSE': 806503.9658264604, 'RMSE': np.float64(898.0556585348485), '

## Comparação de Desempenho das Estratégias de Split

| Método de Split            | Conjunto    | **MAE ↓**     | **MSE ↓**        | **RMSE ↓**    | **MAPE ↓**   |
|----------------------------|------------|--------------|----------------|--------------|-------------|
| **Prequential Expanding**  | Validação  | **748.72**   | **931,445.96** | **965.11**   | **15.30%**  |
|                            | Teste      | **750.69**   | **880,171.31** | **938.17**   | **14.49%**  |
| **Prequential Sliding**    | Validação  | **748.72**   | **931,445.96** | **965.11**   | **15.30%**  |
|                            | Teste      | **750.69**   | **880,171.31** | **938.17**   | **14.49%**  |
| **Expanding Window**       | Validação  | **676.60** ✅ | **611,017.34** ✅ | **781.67** ✅ | **15.08%** ✅ |
|                            | Teste      | **798.65** ❌ | **806,503.97** ✅ | **898.06** ✅ | **18.06%** ❌ |

✅ **Melhores valores em cada métrica estão destacados.**  
❌ **Piores valores em cada métrica estão indicados.**


# Comparação das Estratégias de Validação Cruzada para Séries Temporais

A tabela abaixo compara as três abordagens de **validação cruzada para séries temporais**, considerando diferentes critérios de aplicação e refletindo os resultados obtidos na nova base de dados:

| Critério                                            | Expanding Window Split               | Prequential Sliding Split             | Prequential Expanding Split             |
|-----------------------------------------------------|--------------------------------------|---------------------------------------|-----------------------------------------|
| **Séries temporais reais**                          | ✅ Bom para séries estáveis          | ⚠️ Pode ser arriscado se houver mudanças abruptas  | ✅ Excelente para capturar tendências e sazonalidades  |
| **Aprendizado incremental**                         | ⚠️ O modelo re-treina com todo o histórico, o que pode ser ineficiente | ✅ Indicado para cenários de streaming | ✅ Permite aprendizado incremental com um histórico crescente  |
| **Dados de varejo (sazonais ou promocionais)**      | ⚠️ Pode demandar ajustes finos       | ⚠️ Pode não capturar todo o histórico | ✅ Adapta-se melhor a padrões sazonais e promoções  |
| **Garantia de causalidade**                         | ✅ Sim                              | ⚠️ Se a janela de treino incluir dados do futuro, pode haver vazamento | ✅ Sim                                 |
| **Volume grande de dados**                          | ⚠️ Pode ser mais lento               | ✅ Mais eficiente                     | ⚠️ Pode ser mais custoso computacionalmente  |
| **Previsão futura realista**                        | ✅ Sim                              | ❌ Pode deixar de capturar o contexto completo | ✅ Sim                                 |

---

## Explicação dos Métodos com Base nos Resultados Obtidos

### **Expanding Window Split**
Essa abordagem utiliza um conjunto de treino que cresce progressivamente a cada novo split. Nos resultados apresentados, observamos que o método apresentou **MAE** de **676.60** e **RMSE** de **781.68** na **validação**, e **MAE** de **798.65** e **RMSE** de **898.06** no **teste**.

Embora essa abordagem funcione bem para **séries temporais estáveis**, pode demandar **ajustes finos** para lidar com padrões de varejo, onde há sazonalidades e promoções frequentes. Além disso, como o modelo sempre re-treina com todo o histórico, esse método pode se tornar **mais custoso em termos computacionais**, especialmente com grandes volumes de dados. No entanto, o **Expanding Window garantiu o menor erro na validação**, demonstrando maior estabilidade preditiva.

### **Prequential Sliding Split**
A estratégia de _sliding window_ utiliza uma janela fixa para treino e validação, sendo vantajosa para **aprendizado incremental**, como fluxos contínuos de dados. No entanto, se a janela de treino não for bem definida, pode haver risco de **vazamento de informações do futuro**, violando a causalidade.

Nos testes realizados, os erros foram **MAE ≈ 748.72** e **RMSE ≈ 965.11** na **validação**, e **MAE ≈ 750.69** e **RMSE ≈ 938.17** no **teste**. Apesar disso, essa abordagem pode ser arriscada para séries com mudanças graduais e padrões sazonais bem definidos. Em cenários onde há variações súbitas, essa estratégia pode não ser a melhor escolha, pois pode ignorar padrões sazonais antigos que ainda influenciam a previsão.

### **Prequential Expanding Split**
Essa abordagem combina a vantagem do **crescimento progressivo do histórico** com a capacidade de adaptação a mudanças graduais e padrões sazonais. Isso se torna particularmente relevante para **dados de varejo**, onde ciclos de promoção e sazonalidade afetam diretamente a previsão.

Os resultados indicam um desempenho muito próximo ao _sliding_, com **MAE ≈ 748.72** e **RMSE ≈ 965.11** na **validação**, e **MAE ≈ 750.69** e **RMSE ≈ 938.17** no **teste**. No entanto, o _prequential expanding_ tem a vantagem adicional de **preservar a ordem temporal** e proporcionar uma previsão **mais realista do futuro**. Como o modelo aprende de forma incremental com um **histórico crescente**, essa abordagem **é ideal para capturar tendências e padrões sazonais sem perder informações passadas**.

---

## **Conclusão e Escolha Final**
**Dado o desempenho mais consistente e menor erro na validação, o Expanding Window foi escolhido como a melhor estratégia de validação cruzada.**  

Embora possa ser **mais custoso computacionalmente**, ele **garante menor erro médio e maior estabilidade preditiva**, sendo **a melhor opção para previsões futuras em séries temporais estáveis**. Em um ambiente de varejo, ajustes finos podem ser necessários, mas a vantagem de **preservar padrões históricos e garantir causalidade** faz dessa a abordagem mais confiável.

**Essa escolha, vai garantir um modelo mais robusto e estável para previsão em séries temporais!**

